In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE silver_revenue
COMMENT "Cleaned Zomato revenue data with consistent schema"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
    CAST(OrderID AS STRING) AS OrderID,
    CAST(OrderDate AS DATE) AS OrderDate,
    City,
    Restaurant,
    Category,
    CAST(ItemTotal AS DECIMAL(10,2)) AS ItemTotal,
    CAST(DeliveryFee AS DECIMAL(10,2)) AS DeliveryFee,
    CAST(Taxes AS DECIMAL(10,2)) AS Taxes,
    CAST(Discount AS DECIMAL(10,2)) AS Discount,
    PromoCodeUsed,
    CAST(TotalRevenue AS DECIMAL(10,2)) AS TotalRevenue,
    input_file_path,
    ingestion_timestamp
FROM STREAM(bronze_revenue)
WHERE
    OrderID IS NOT NULL
    AND OrderDate IS NOT NULL
    AND City IS NOT NULL
    AND Restaurant IS NOT NULL
    AND Category IS NOT NULL
    AND ItemTotal IS NOT NULL
    AND DeliveryFee IS NOT NULL
    AND Taxes IS NOT NULL
    AND Discount IS NOT NULL
    AND TotalRevenue IS NOT NULL;

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE silver_customers
COMMENT "Cleaned Zomato customer data with consistent schema"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
    CAST(FullName AS STRING) AS full_name,
    CAST(CustomerId AS STRING) AS customer_id,
    CAST(Gender AS STRING) AS gender,
    CAST(City AS STRING) AS city,
    CAST(Age AS INT) AS age,
    CAST(Customer_Segment AS STRING) AS customer_segment,
    CAST(signup_date AS DATE) AS signup_date,
    CAST(total_orders AS INT) AS total_orders,
    CAST(total_spent AS DECIMAL(10,2)) AS total_spent,
    input_file_path,
    ingestion_timestamp
FROM STREAM(bronze_customers)
WHERE
    FullName IS NOT NULL
    AND CustomerId IS NOT NULL
    AND Gender IS NOT NULL
    AND City IS NOT NULL
    AND Age IS NOT NULL
    AND Customer_Segment IS NOT NULL
    AND signup_date IS NOT NULL
    AND total_orders IS NOT NULL
    AND total_spent IS NOT NULL;


In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE silver_menuitems
COMMENT "Cleaned Zomato menu item data with consistent schema"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
    CAST(ItemId AS STRING) AS ItemId,
    CAST(RestaurantId AS STRING) AS RestaurantId,
    CAST(Item_Name AS STRING) AS ItemName,
    CAST(Category AS STRING) AS Category,
    CAST(Price AS DECIMAL(10,2)) AS Price,
    CAST(Is_Veg AS BOOLEAN) AS Is_Veg,
    input_file_path,
    ingestion_timestamp
FROM STREAM(bronze_menuitems)
WHERE
    ItemId IS NOT NULL
    AND RestaurantId IS NOT NULL
    AND Item_Name IS NOT NULL
    AND Category IS NOT NULL
    AND Price IS NOT NULL
    AND Is_Veg IS NOT NULL;


In [0]:
-- Simple, streaming-safe Silver for order items
CREATE OR REFRESH STREAMING TABLE silver_orderItems
COMMENT "Cleaned Zomato order items data with consistent schema (streaming-safe)"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  UPPER(TRIM(CAST(OrderItemId AS STRING))) AS OrderItemId,
  UPPER(TRIM(CAST(OrderId      AS STRING))) AS OrderId,
  UPPER(TRIM(CAST(ItemId       AS STRING))) AS ItemId,

  CAST(Quantity   AS INT)           AS Quantity,
  CAST(Unit_Price AS DECIMAL(10,2)) AS Unit_Price,

  CAST(
    COALESCE(
      NULLIF(Line_Total, ''),
      CAST(Quantity AS DECIMAL(12,2)) * CAST(Unit_Price AS DECIMAL(12,2))
    )
    AS DECIMAL(12,2)
  ) AS Line_Total,

  input_file_path,
  ingestion_timestamp
FROM STREAM(bronze_orderItems)
WHERE
  -- Required fields present
  OrderItemId IS NOT NULL
  AND OrderId IS NOT NULL
  AND ItemId IS NOT NULL
  AND Quantity IS NOT NULL
  AND Unit_Price IS NOT NULL

  -- Basic quality checks
  AND Quantity > 0
  AND Unit_Price >= 0

  -- Ensure line total is present (after derivation) and consistent within tolerance
  AND CAST(
        COALESCE(
          NULLIF(Line_Total, ''),
          CAST(Quantity AS DECIMAL(12,2)) * CAST(Unit_Price AS DECIMAL(12,2))
        )
      AS DECIMAL(12,2)
      ) IS NOT NULL
  AND ABS(
        CAST(
          COALESCE(
            NULLIF(Line_Total, ''),
            CAST(Quantity AS DECIMAL(12,2)) * CAST(Unit_Price AS DECIMAL(12,2))
          )
        AS DECIMAL(12,2)) 
        - (CAST(Quantity AS DECIMAL(12,2)) * CAST(Unit_Price AS DECIMAL(12,2)))
      ) <= 0.01;

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE silver_operations
COMMENT "Cleaned Zomato operations data with consistent schema"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
    CAST(OrderId AS STRING) AS OrderId,
    CAST(PartnerId AS STRING) AS PartnerId,
    CAST(City AS STRING) AS City,
    CAST(`Delivery_time` AS INT) AS Delivery_time,
    CAST(`Delivery_distance` AS DECIMAL(10,2)) AS Delivery_distance,
    CAST(Weather AS STRING) AS Weather,
    CAST(Pickuptime AS TIMESTAMP) AS Pickuptime,
    CAST(Delivered_time AS TIMESTAMP) AS Delivered_time,
    input_file_path,
    ingestion_timestamp
FROM STREAM(bronze_operations)
WHERE
    OrderId IS NOT NULL
    AND PartnerId IS NOT NULL
    AND City IS NOT NULL
    AND `Delivery_time` IS NOT NULL
    AND `Delivery_distance` IS NOT NULL
    AND Weather IS NOT NULL
    AND Pickuptime IS NOT NULL
    AND Delivered_time IS NOT NULL;


In [0]:
CREATE OR REFRESH STREAMING TABLE silver_restaurants
COMMENT "Cleaned Zomato restaurant data with consistent schema"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
    CAST(RestaurantId AS STRING) AS RestaurantId,
    Restaurant_Name,
    City,
    CAST(Rating AS DECIMAL(3,2)) AS Rating,
    Cuisine_type,
    CAST(Costfortwo AS DECIMAL(10,2)) AS Costfortwo,
    CAST(AverageDeliverytime AS INT) AS AverageDeliverytime,
    Restaurant_type,
    input_file_path,
    ingestion_timestamp
FROM STREAM(bronze_restaurants)
WHERE
    RestaurantId IS NOT NULL
    AND Restaurant_Name IS NOT NULL
    AND City IS NOT NULL
    AND Rating IS NOT NULL
    AND Cuisine_type IS NOT NULL
    AND Costfortwo IS NOT NULL
    AND AverageDeliverytime IS NOT NULL
    AND Restaurant_type IS NOT NULL;

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_orders
COMMENT "Cleaned Zomato order data with consistent schema"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
    CAST(OrderId AS STRING) AS OrderId,
    CAST(CustomerId AS STRING) AS CustomerId,
    CAST(RestaurantId AS STRING) AS RestaurantId,
    CAST(PartnerId AS STRING) AS PartnerId,
    CAST(Order_amount AS DECIMAL(10,2)) AS Order_amount,
    CAST(tax AS DECIMAL(10,2)) AS tax,
    CAST(discount AS DECIMAL(10,2)) AS discount,
    CAST(delivery_fee AS DECIMAL(10,2)) AS delivery_fee,
    CAST(total_food_items AS INT) AS total_food_items,
    CAST(order_datetime AS TIMESTAMP) AS order_datetime,
    payment_method,
    order_status,
    promo_code_used,
    promo_code,
    City,
    input_file_path,
    ingestion_timestamp
FROM STREAM(bronze_orders)
WHERE
    OrderId IS NOT NULL
    AND CustomerId IS NOT NULL
    AND RestaurantId IS NOT NULL
    AND PartnerId IS NOT NULL
    AND Order_amount IS NOT NULL
    AND tax IS NOT NULL
    AND discount IS NOT NULL
    AND delivery_fee IS NOT NULL
    AND total_food_items IS NOT NULL
    AND order_datetime IS NOT NULL
    AND payment_method IS NOT NULL
    AND order_status IS NOT NULL
    AND City IS NOT NULL;
